In [28]:
from dotenv import load_dotenv
from openai import OpenAI
import os
import json
import requests
from PyPDF2 import PdfReader
import gradio as gr

In [11]:
load_dotenv(override=True)
openai = OpenAI()

In [34]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"

In [13]:
def push(message):
    print(f"Push: ${message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [14]:
push("Hola amigooo!!!")

Push: $Hola amigooo!!!


In [ ]:
def record_user_details(email , name="Name not provided" , notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [ ]:
def record_unknown_question(question):
    push(f"Recording {question} asked that I coundn't answer")
    return {"recorded": "ok" }

In [19]:
record_user_details_json = {
    "name": "record_user_details",
    "description": "Use this tool to record that a user is interested in being in touch and provided an email address",
    "parameters": {
        "type": "object",
        "properties": {
            "email": {
                "type": "string",
                "description": "The email address of this user"
            },
            "name": {
                "type": "string",
                "description": "The user's name, if they provide it"
            },
            "notes": {
                "type": "string",
                "description": "Any additional information about the conversation that is worth recording to give context"
            }
        },
        "required": ["email"],
        "additionalProperties": False
    }
}


In [20]:
record_unknown_question_json = {
    "name": "record_unknown_question",
    "description": "Always use this tool to record questions that could not be answered as you don't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type" : "string",
                "description" : "The question that could not be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function" , "function": record_user_details_json}, 
         {"type": "function" , "function": record_unknown_question_json}]  

In [ ]:
def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)
        tool = globals().get(tool_name)
        result = tool(**arguments) if tool else {}
        results.append({"role": "tool", "content": json.dumps(result) , "tool_call_id": tool_call.id})
    return results

In [24]:
globals()["record_unknown_question"]("This is a really hard question")

Push: $Recording This is a really hard question


{'recorded': 'ok'}

In [29]:
reader = PdfReader("Ahmed-Saber-CV.pdf")
data = ""
for page in reader.pages:
    text = page.extract_text()
    if text:
        data += text

reader = PdfReader("GitHub skills extraction.pdf")
github_skills = "github skills:\n"
for page in reader.pages:
    text = page.extract_text()
    if text:
        github_skills += text

In [31]:
name = "Ahmed Saber"

In [ ]:
system_prompt = f"You are acting as {name}. You are answering questions on {name}'s website, \
particularly questions related to {name}'s career, background, skills and experience. \
Your responsibility is to represent {name} for interactions on the website as faithfully as possible. \
You are given a summary of {name}'s background and github repositories content and projects which you can use to answer questions. \
Be professional and engaging, as if talking to a potential client or future employer who came across the website. \
If you don't know the answer to any question, use your record_unknown_question tool to record the question that you couldn't answer, even if it's about something trivial or unrelated to career. \
If the user is engaging in discussion, try to steer them towards getting in touch via email; ask for their email and record it using your record_user_details tool. "

system_prompt += f"\n\n## Summary:\n{data}\n\n## github:\n{github_skills}\n\n"
system_prompt += f"With this context, please chat with the user, always staying in character as {name}."


In [ ]:
## Build chat function with handling tool calls